In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'flower-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2955119%2F5088988%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240620%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240620T113954Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D5679af724fddace955c42ceeb40560570f5cddbd97cab0df4a4187799cd90532c32340b5ecdb5f054b52935008d809d762e5a6af57382a7e47b389c998ba6e7949f3d3e0ce3e09157ac8e28d77801adfb5bfaa03921fdaab1e80f0d1446476c185e80fa82a5abe13e41a144fbbd2c990e98b67a09300a7a564ec1fad83b80a8aa5d358bc6a4aa270c531bef3f49a3883ae42a8af20f3ed412e10593acf61d2d13448930892e8478d0b17a03b0f68051ce1a91b3574a884e61439f72fbf03f16dddfb4f315643da1ec670fe79b2033237c607ed9ff837ff9a2e7215597d8a06947a8a7684daba533aaa3e958b3ab0ce117cb5916bcae9494e39d0ceb9ef372c9a'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 346508153 bytes downloaded
Downloaded and uncompressed: flower-dataset
Data source import complete.


In [ ]:
# Importing required packages

import os
import json
import torch

import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt

from torch import nn
from torch import optim
from torchvision import datasets, transforms, models
from PIL import Image
# Set up matplotlib for inline plotting with high-resolution figures
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
# Hyperparameters
batch_size = 64
epochs = 10
learning_rate = 0.001
criterion = nn.NLLLoss()

# Other constants
input_size = 25088
output_size = 102
hidden_layers = 512
drop_rate = 0.2

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Set the directory paths for training, validation, and testing data
data_dir = '/kaggle/input/flower-dataset/flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'
# Creating a normalization transform that adjusts image values to have a standard mean and deviation
normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)

data_transforms = {}

# For training data: Randomly crop and resize to 224x224, then convert to tensor and normalize
data_transforms['train'] = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(.5), # randomly flips images horizontally with a probability of 0.5 (50% chance).
    transforms.RandAugment(num_ops=2, magnitude=15), #  operations like color adjustments, rotations, translations, shears, and more.
    transforms.ToTensor(),
    normalize
])

# For validation data: Randomly crop and resize to 224x224, then convert to tensor and normalize
data_transforms['val'] = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.ToTensor(),
    normalize
])

# For test data: Randomly crop and resize to 224x224, then convert to tensor and normalize
data_transforms['test'] = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.ToTensor(),
    normalize
])
# Load the datasets using ImageFolder, a generic data loader where the images are arranged in this way by defa
image_datasets = {
    'train': datasets.ImageFolder(train_dir, transform=data_transforms['train']),
    'val': datasets.ImageFolder(valid_dir, transform=data_transforms['val']),
    'test': datasets.ImageFolder(test_dir, transform=data_transforms['test'])
}

# Create dataloaders for the datasets
dataloaders = {
    'train': torch.utils.data.DataLoader(image_datasets['train'], batch_size, shuffle=True),
    'val': torch.utils.data.DataLoader(image_datasets['val'], batch_size, shuffle=True),
    'test': torch.utils.data.DataLoader(image_datasets['test'], batch_size=32, shuffle=True)
}
# A mapping of category names to labels
cat_to_name = {"21": "fire lily", "3": "canterbury bells", "45": "bolero deep blue", "1": "pink primrose", "34": "mexican aster", "27": "prince of wales feathers", "7": "moon orchid", "16": "globe-flower", "25": "grape hyacinth", "26": "corn poppy", "79": "toad lily", "39": "siam tulip", "24": "red ginger", "67": "spring crocus", "35": "alpine sea holly", "32": "garden phlox", "10": "globe thistle", "6": "tiger lily", "93": "ball moss", "33": "love in the mist", "9": "monkshood", "102": "blackberry lily", "14": "spear thistle", "19": "balloon flower", "100": "blanket flower", "13": "king protea", "49": "oxeye daisy", "15": "yellow iris", "61": "cautleya spicata", "31": "carnation", "64": "silverbush", "68": "bearded iris", "63": "black-eyed susan", "69": "windflower", "62": "japanese anemone", "20": "giant white arum lily", "38": "great masterwort", "4": "sweet pea", "86": "tree mallow", "101": "trumpet creeper", "42": "daffodil", "22": "pincushion flower", "2": "hard-leaved pocket orchid", "54": "sunflower", "66": "osteospermum", "70": "tree poppy", "85": "desert-rose", "99": "bromelia", "87": "magnolia", "5": "english marigold", "92": "bee balm", "28": "stemless gentian", "97": "mallow", "57": "gaura", "40": "lenten rose", "47": "marigold", "59": "orange dahlia", "48": "buttercup", "55": "pelargonium", "36": "ruby-lipped cattleya", "91": "hippeastrum", "29": "artichoke", "71": "gazania", "90": "canna lily", "18": "peruvian lily", "98": "mexican petunia", "8": "bird of paradise", "30": "sweet william", "17": "purple coneflower", "52": "wild pansy", "84": "columbine", "12": "colt's foot", "11": "snapdragon", "96": "camellia", "23": "fritillary", "50": "common dandelion", "44": "poinsettia", "53": "primula", "72": "azalea", "65": "californian poppy", "80": "anthurium", "76": "morning glory", "37": "cape flower", "56": "bishop of llandaff", "60": "pink-yellow dahlia", "82": "clematis", "58": "geranium", "75": "thorn apple", "41": "barbeton daisy", "95": "bougainvillea", "43": "sword lily", "83": "hibiscus", "78": "lotus lotus", "88": "cyclamen", "94": "foxglove", "81": "frangipani", "74": "rose", "89": "watercress", "73": "water lily", "46": "wallflower", "77": "passion flower", "51": "petunia"}
# Load a pre-trained VGG16 model
model = models.vgg16(pretrained=True)

for param in model.parameters():
    param.requires_grad = False
    import torch.nn as nn
import torch.nn.functional as F

class MyNetwork(nn.Module):
    """
    A simple feedforward neural network model.

    Args:
        input_size (int): Number of input features.
        hidden_units (int): Number of hidden units in the layers.
        drop_rate (float): Dropout rate for regularization.
        output_size (int): Number of output classes.

    Attributes:
        layer1 (nn.Linear): First fully connected layer.
        layer2 (nn.Linear): Second fully connected layer.
        layer3 (nn.Linear): Third fully connected layer.
        dropout (nn.Dropout): Dropout layer for regularization.
        output (nn.LogSoftmax): Log-softmax activation for classification.
    """
    def __init__(self, input_size, hidden_units, drop_rate, output_size):
        super().__init__()

        self.layer1 = nn.Linear(input_size, hidden_units)
        self.layer2 = nn.Linear(hidden_units, hidden_units // 2)
        self.layer3 = nn.Linear(hidden_units // 2, output_size)

        self.dropout = nn.Dropout(drop_rate)

        self.output = nn.LogSoftmax(dim=1)

    def forward(self, x):
        """
        Forward pass through the network.

        Args:
            x (torch.Tensor): Input tensor.

        Returns:
            torch.Tensor: Output tensor after passing through the network.
        """
        out = self.layer1(x)
        out = F.relu(out)
        out = self.dropout(out)

        out = self.layer2(out)
        out = F.relu(out)
        out = self.dropout(out)

        out = self.layer3(out)

        out = self.output(out)

        return out
model.classifier = MyNetwork(input_size, hidden_layers, drop_rate, output_size)
import torch

def get_validation(model, device, loaders, criterion):
    """
    Calculate loss and accuracy on the validation dataset.

    Args:
        model (torch.nn.Module): The neural network model.
        device (torch.device): The device (CPU or GPU) for computation.
        dataloaders (dict): Dictionary containing data loaders for training and validation.
        criterion (torch.nn.Module): The loss function.

    Returns:
        tuple: A tuple containing loss and accuracy values.
    """
    loss, accuracy = 0, 0

    model.to(device)

    for images, labels in loaders:
        images, labels = images.to(device), labels.to(device)
        output = model(images)
        loss += criterion(output, labels).detach()*images.size(0)
        ps = torch.exp(output)
        equity = labels == ps.max(dim=1)[1]
        accuracy += equity.type(torch.FloatTensor).mean()

    return loss / len(loaders.dataset), 100 * accuracy / len(loaders)
def train_network():
    """
    Train the network using the specified data loaders and settings.

    Returns:
        torch.nn.Module: The trained model.
    """
    trainloader = dataloaders["train"]

    model.to(device)

    optimizer = optim.Adam(model.classifier.parameters(), lr=learning_rate)  # Use Adam optimizer for model's classifier parameters


    for epoch in range(epochs):
        training_loss = 0
        for images, labels in trainloader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()

            output = model(images)

            Loss = criterion(output, labels)

            Loss.backward()

            optimizer.step()

            training_loss += Loss.item()

        model.eval()
        with torch.no_grad():
            loss, accuracy = get_validation(model, device, dataloaders["val"], criterion)

        print(
            f"Epoch: {epoch + 1}/{epochs}\n"
            f"Training Loss: {training_loss / len(dataloaders['train']):.4f}\n"
            f"Validation Loss: {loss / len(dataloaders['val']):.4f}\n"
            f"Validation Accuracy: {accuracy:.2f}%\n\n"
        )

        model.train()

    return model
# Trains a neural network and returns the trained model
trained_model = train_network()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:02<00:00, 214MB/s]


Epoch: 1/10
Training Loss: 3.4523
Validation Loss: 0.1433
Validation Accuracy: 51.66%


Epoch: 2/10
Training Loss: 2.1601
Validation Loss: 0.0960
Validation Accuracy: 64.40%


Epoch: 3/10
Training Loss: 1.8543
Validation Loss: 0.0851
Validation Accuracy: 69.52%


Epoch: 4/10
Training Loss: 1.6544
Validation Loss: 0.0766
Validation Accuracy: 71.32%


Epoch: 5/10
Training Loss: 1.5773
Validation Loss: 0.0728
Validation Accuracy: 74.46%


Epoch: 6/10
Training Loss: 1.4867
Validation Loss: 0.0679
Validation Accuracy: 74.56%


Epoch: 7/10
Training Loss: 1.4750
Validation Loss: 0.0622
Validation Accuracy: 80.19%


Epoch: 8/10
Training Loss: 1.3931
Validation Loss: 0.0609
Validation Accuracy: 79.13%


Epoch: 9/10
Training Loss: 1.3685
Validation Loss: 0.0574
Validation Accuracy: 79.62%


Epoch: 10/10
Training Loss: 1.2570
Validation Loss: 0.0533
Validation Accuracy: 81.03%


